<a href="https://colab.research.google.com/github/axel-sirota/test-teach-platzi/blob/main/Template_TestTeachPlatzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning de Transformers con HuggingFace

Author: Axel Sirota

HuggingFace es una empresa con una fuerte filosofía de código abierto que hace que los Transformers estén fácilmente disponibles para que no tenga que hacer lo que hicimos antes para cada aplicación.

## Prep

In [ ]:
!pip install -U datasets evaluate transformers transformers[sentencepiece]

In [ ]:
import multiprocessing
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np

import sys
import keras.backend as K
import random
import os
import pandas as pd
import warnings
import time


TRACE = False
EPOCHS = 2
BATCH_SIZE = 256

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')

## Tokenizar y cargar el Dataset

En HuggingFace hay muchos modelos, y cada uno tiene su propio tokenizador. Por suerte para nosotros, hay una clase `AutoTokenizer` que hace el trabajo pesado después de que proporcionamos el checkpoint.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

raw_datasets = load_dataset("imdb")  # loads dataset raw
raw_datasets

Notemos que es un diccionario con los datasets de entrenamiento, evaluacion y no supervisado.

In [ ]:
raw_datasets['train'][0]  # Let's see the first review

Pero como sabemos si label 0 significa positivo o negativo? Tenemos que ver el atributo features

In [ ]:
label = raw_datasets['train'].features['label']
label.int2str(0)

Ahí está, dentro de características vemos que el índice 0 es **Negativo**

Ahora, para tokenizar el conjunto de datos, necesitamos cargar el tokenizador adecuado para el modelo que nos interesa. ¡Y la vamos a aplicar en todas partes!

Después de este paso, el tokenizador convierte el texto en un tensor de identificadores, cada uno de los cuales representa una palabra diferente en el vocabulario BERT.

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    # We are using the BERT tokenizer, specifying to PAD until the end,
    # truncate if either 128 elements are met or the maximum from the model, which you get from the model card

    return tokenizer(example["text"], padding=True, truncation=True, max_length=128)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


Let's see how it worked!

In [ ]:
tokenized_datasets['train'][0]['text']

In [ ]:
tokenizer(tokenized_datasets['train'][0]['text'])

El tokenizador de BERT (bueno, DistillBERT) convierte cada palabra en su ID de acuerdo con *su* vocabulario. Y observe que el enmascaramiento dice que no hemos sido truncados. Lo que sí sabremos es hacer esto para todos los datos y convertirlos en un objeto TF Datasets (que Keras acepta)

In [ ]:

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=['input_ids'],
    label_cols=["label"],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=['input_ids'],
    label_cols=["label"],
    shuffle=False,
    batch_size=BATCH_SIZE,
)

In [ ]:
for inputs, labels in tf_train_dataset.take(1):
  print(f' inputs: {inputs.shape}, labels: {labels.shape}')


## Descargar el modelo y prepararse para el entrenamiento

Ahora vamos a descargar el modelo. Es muy importante que utilice la clase que comienza con `TFAutoModel`. Hay modelos automáticos para la mayoría de las tareas, por lo que no tiene que agregar manualmente el encabezado, por ejemplo, `TFAutoModelForSequenceClassification` agrega una capa Densa (SIN SOFTMAX) para hacer la clasificación

Ahora vamos a definir el optimizador y la funcion de perdida. Es muy important que la Perdida tenga `from_logits=True` porque los modelos de Hugging Face devuelven logits, no probabilidades.

Compilamos el modelo

Y veamos el summary del modelo

¡Oh, no! ¡Tenemos demasiados parámetros para entrenar! Por suerte en Keras es muy fácil configurar algunas capas como no entrenables

*Voilá!*

Entrenemos el modelo!

Ahora tenemos un modelo entrenado que transfirió el aprendizaje de DistillBERT

## Probando el modelo

Primero probemos con dos oraciones, una negativa y una positiva. Vamos a tener que tokenizarlas que hicimos antes

Vamos a analizar esos tokens

Podemos ver que la primera oracion tuvo padding, como esperabamos. Vamos a predecir con estos input_ids

Notemos que el output de la predicciónson logits, no probabilidades. Por lo que conviene aqui aplicar `tf.math.softmax` al resultado.

Perfecto, ahora si hacemos un `tf.math.argmax` vamos a obtener que clase es.

Para recordar las clases tenemos la variable `label` que preserva los indices del output de la softmax

El modelo esta correcto!

Si quisiesemos publicar el modelo primero evaluemoslo con el evaluation set